In [ ]:
!pip install datasets rouge_score openai datetime numpy

In [4]:
def count_text_tokens_default(text):
    import re
    l = re.findall(r"[\w]+|[’+-−.,!?:;@#\$%\'\"\^&*_=\\|/<>\[\]\(\)\n\t]", text)
    return len(l), l

In [36]:
from datasets import load_dataset, load_metric

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')
xsum_dataset = load_dataset("xsum")
rouge = load_metric("rouge")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [127]:
import tqdm 
import numpy as np
import math

def print_rouge_score(generated, reference):
    rouge.add_batch(predictions=generated, references=reference)
    results = rouge.compute()
    l_rouge = [[0,0,0],[0,0,0],[0,0,0]]
    l_rouge[0][0] = results['rouge1'].mid.recall
    l_rouge[0][1] = results['rouge1'].mid.precision
    l_rouge[0][2] = results['rouge1'].mid.fmeasure
    l_rouge[1][0] = results['rouge2'].mid.recall
    l_rouge[1][1] = results['rouge2'].mid.precision
    l_rouge[1][2] = results['rouge2'].mid.fmeasure
    l_rouge[2][0] = results['rougeL'].mid.recall
    l_rouge[2][1] = results['rougeL'].mid.precision
    l_rouge[2][2] = results['rougeL'].mid.fmeasure
    
    print('rouge-1: r:%2.2f, p:%2.2f f:%2.2f' % (np.mean(l_rouge[0][0]), np.mean(l_rouge[0][1]), np.mean(l_rouge[0][2])))
    print('rouge-2: r:%2.2f, p:%2.2f f:%2.2f' % (np.mean(l_rouge[1][0]), np.mean(l_rouge[1][1]), np.mean(l_rouge[1][2])))
    print('rouge-l: r:%2.2f, p:%2.2f f:%2.2f' % (np.mean(l_rouge[2][0]), np.mean(l_rouge[2][1]), np.mean(l_rouge[2][2])))

import openai 

openai.api_key = ""
openai.api_base =  "https://imr-azure-openai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2022-12-01'
deployment_id='azure-openai-text-davinci-003' 

def summarize_with_openai(text, max_length, extreme = False):
    prompt = 'summarize the text above is less than ' + str(max_length) + ' words.'
    if extreme:
        prompt = 'extreme ' + prompt
    response = openai.Completion.create(engine=deployment_id, prompt= text +'\n' + prompt , max_tokens=1000)
    return response['choices'][0]['text'].split("\n")[-1]

def ask_question(text):
    response = openai.Completion.create(engine=deployment_id, prompt=question, max_tokens=3000)
    print(response)
    
def test_dataset(dataset, subset, keys, batch_num=math.inf, extreme = False):
    from torch.utils.data import DataLoader, RandomSampler

    #dslen = dataset[subset].num_rows
    
    test_sampler = RandomSampler(dataset[subset])
    test_dataloader = DataLoader(dataset[subset], sampler=test_sampler, batch_size=128)
    test_iter = iter(test_dataloader)
    #sample_list = list(zip(sample[key[0]], sample[key[1]]))
    batch_num = min(len(test_iter), batch_num)
    generated = []
    references = []
    for i, batch in tqdm.tqdm(enumerate(test_iter), total=batch_num):
        sample_list = list(zip(batch[keys[0]], batch[keys[1]]))
        for doc, ref in sample_list:
            try:
                ref_length, _ = count_text_tokens_default(ref)
                generated.append(summarize_with_openai(doc, ref_length, extreme))
                references.append(ref)
            except Exception as e:
                print(e)
        print(i, batch_num)
        if ((i + 1) == batch_num):
            break
    
    print_rouge_score(generated, references)


# Test ask funcitons

In [27]:
question = """
k-folds cross validation processes entails?
"""
ask_question(question)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nK-folds cross-validation is a process used to evaluate a model\u2019s performance on unseen data. It involves randomly splitting the dataset into k equal sized \u2018folds\u2019. Each fold is used as a testing set once and as a training set k-1 times. The average performance across all k folds is then used to judge the model\u2019s performance. This process helps avoid overfitting or underfitting the model."
    }
  ],
  "created": 1675767994,
  "id": "cmpl-6hG5yjnmkDtXydYDqVcvKzSzhF6fD",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 93,
    "prompt_tokens": 10,
    "total_tokens": 103
  }
}


In [105]:
print(cnn_dataset['test']['article'][0])
print("\nGenerated:")
print(summarize_with_openai(cnn_dataset['test']['article'][0], 120))
print("\nReference:")
print(cnn_dataset['test']['highlights'][0])

(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, sa

In [110]:
print(xsum_dataset['test']['document'][0])
print("\nGenerated:")
print(summarize_with_openai(xsum_dataset['test']['document'][0], 30, true))
print("\nReference:")
print(xsum_dataset['test']['summary'][0])

Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for accommodat

# Run ChatGTP to summarize XSum and CNN/DM test sets

In [93]:
import datetime
start = datetime.datetime.utcnow()

test_dataset(xsum_dataset, "test", ["document", "summary"])

end = datetime.datetime.utcnow()
print("\nDuration =", end-start)

  0%|          | 0/89 [00:00<?, ?it/s]

This model's maximum context length is 4097 tokens, however you requested 5614 tokens (5598 in your prompt; 16 for the completion). Please reduce your prompt; or completion length.


  7%|▋         | 6/89 [14:35<3:21:32, 145.69s/it]

This model's maximum context length is 4097 tokens, however you requested 4119 tokens (4097 in your prompt; 22 for the completion). Please reduce your prompt; or completion length.


 42%|████▏     | 37/89 [1:08:50<1:19:25, 91.65s/it]

...

This model's maximum context length is 4097 tokens, however you requested 15312 tokens (15280 in your prompt; 32 for the completion). Please reduce your prompt; or completion length.
Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 89%|████████▉ | 79/89 [2:13:16<14:51, 89.11s/it]

This model's maximum context length is 4097 tokens, however you requested 4574 tokens (4533 in your prompt; 41 for the completion). Please reduce your prompt; or completion length.


100%|██████████| 89/89 [2:27:26<00:00, 99.40s/it]


rouge-1: r:0.28, p:0.27 f:0.28
rouge-2: r:0.08, p:0.08 f:0.08
rouge-l: r:0.21, p:0.21 f:0.21

Duration = 2:27:36.396431


In [126]:
import datetime
start = datetime.datetime.utcnow()

test_dataset(xsum_dataset, "test", ["document", "summary"], 1, True)

end = datetime.datetime.utcnow()
print("\nDuration =", end-start)

100%|██████████| 1/1 [02:28<00:00, 148.56s/it]

0 1


2it [04:58, 149.33s/it]                       

1 1


2it [07:31, 225.56s/it]

2 1


rouge-1: r:0.25, p:0.23 f:0.24
rouge-2: r:0.06, p:0.06 f:0.06
rouge-l: r:0.19, p:0.17 f:0.18

Duration = 0:07:31.672131


In [128]:
import datetime
start = datetime.datetime.utcnow()

test_dataset(cnn_dataset, "test", ["article", "highlights"], 1)

end = datetime.datetime.utcnow()
print("\nDuration =", end-start)

  0%|          | 0/1 [04:36<?, ?it/s]

0 1


rouge-1: r:0.41, p:0.35 f:0.37
rouge-2: r:0.15, p:0.13 f:0.14
rouge-l: r:0.28, p:0.24 f:0.25

Duration = 0:04:36.682141


In [94]:
import datetime
start = datetime.datetime.utcnow()

test_dataset(cnn_dataset, "test", ["article", "highlights"])

end = datetime.datetime.utcnow()
print("\nDuration =", end-start)

100%|██████████| 90/90 [4:45:52<00:00, 190.59s/it]  


rouge-1: r:0.42, p:0.41 f:0.41
rouge-2: r:0.17, p:0.17 f:0.17
rouge-l: r:0.28, p:0.27 f:0.27

Duration = 4:46:29.570380


# Test rouge on random generated strings out of stop words

In [5]:
def generate_random_sequence(words, text_len=1):
    import random
    import math
    l = len(words)
    if text_len < 1:
        return ''
    txt = words[math.floor(random.random() * l)]
    i = 1
    while i < text_len:
        txt += ' ' + words[math.floor(random.random() * l)]
        i += 1
    return txt

def test_dataset_random(dataset, key1, key2, words):
    generated = []
    references = []

    for x in dataset[key1]:
        ref_len, _ = count_text_tokens_default(x[key2])
        generated.append(generate_random_sequence(words, ref_len))
        references.append(x[key2])
    
    print_rouge_score(generated, references)

In [6]:
print('cnndm dataset:')
test_dataset_random(cnn_dataset, 'test', 'highlights', ['of', 'to', 'the'])

cnndm dataset:
rouge-1: r:0.08, p:0.07 f:0.08
rouge-2: r:0.00, p:0.00 f:0.00
rouge-l: r:0.08, p:0.07 f:0.08


In [7]:
print('xsum dataset:')
test_dataset_random(xsum_dataset, 'test', 'summary', ['of', 'to', 'the'])

xsum dataset:
rouge-1: r:0.10, p:0.09 f:0.09
rouge-2: r:0.01, p:0.01 f:0.01
rouge-l: r:0.10, p:0.09 f:0.09
